### Load Dependencies

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  

from netCDF4 import Dataset, num2date, date2index
from datetime import datetime

import time as tm
import numpy as np
import numpy.ma as ma
from numpy import argmin, linspace, size, array

from cdo import *
cdo = Cdo()

import matplotlib.pyplot as plt
import matplotlib.path as mpath
import matplotlib.colors as colors
import matplotlib.patches as mpatches
from matplotlib.pyplot import cm
from matplotlib.gridspec import GridSpec
from matplotlib.colors import ListedColormap

import cartopy as cp
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker

from shapely import geometry
from collections import namedtuple
from shapely.geometry.polygon import LinearRing

import seaborn as sns
import xarray as xr
import pandas as pd

In [2]:
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

colors = ['midnightblue','mediumblue','blue','dodgerblue','deepskyblue','lightskyblue','white','yellow','orange','orangered','red','firebrick','purple']
cmap1 = LinearSegmentedColormap.from_list("mycmap", colors)
nodes = [0, 0.1, 0.35,0.4,0.45,0.475,0.5,0.525,0.55,0.60,0.65,0.9, 1.0]
cmap2 = LinearSegmentedColormap.from_list("mycmap", list(zip(nodes, colors)))

### Obs-based Ensemble Mean

In [3]:
f1 = xr.open_dataset('/work/uo1075/from_Mistral/uo1075/u301331/observations/ens_1990-2020_co2.nc')
f2 = f1.co2flux.sel(lon=slice(-180,180),lat=slice(-65,-35)).sel(time=slice('1990','2019'))

#Mean Fluxes
obs_yr = f2.groupby('time.year').mean(dim='time').mean(dim='year')

seasons_obs = f2.groupby('time.season').mean(dim='time')
obs_DJF = seasons_obs.sel(season='DJF')/4
obs_MAM = seasons_obs.sel(season='MAM')/4
obs_JJA = seasons_obs.sel(season='JJA')/4
obs_SON = seasons_obs.sel(season='SON')/4

In [4]:
observations=['lsce','csir','spco2','jena','jma','nies','ens']
f_ens = xr.open_dataset('/work/uo1075/from_Mistral/uo1075/u301331/observations/ens_1990-2020_co2.nc')
f_ens_time = f_ens.time

co2_ym=[]
co2_win=[]
co2_spr=[]
co2_sum=[]
co2_aut=[]

for i in range(0,len(observations)):
    f = '{}{}{}'.format('/work/uo1075/from_Mistral/uo1075/u301331/observations/',observations[i],'_1990-2020_co2.nc')
    # print(observations[i])
        
    f1 = xr.open_dataset(f, decode_times=False)
    if (i == 3):
        f1= f1.rename({'mtime': 'time'})
    f1['time'] = f_ens_time
    
    f2 = f1.co2flux.sel(lon=slice(-180,180),lat=slice(-65,-35)).sel(time=slice('1990','2019'))
    co2_ym.append((f2.groupby('time.year').mean('time')).sum(('lon','lat')))
    co2_win.append(f2.sel(time=f2.time.dt.season=="DJF").groupby('time.year').mean('time').sum(('lon','lat')))
    co2_spr.append(f2.sel(time=f2.time.dt.season=="MAM").groupby('time.year').mean('time').sum(('lon','lat')))
    co2_sum.append(f2.sel(time=f2.time.dt.season=="JJA").groupby('time.year').mean('time').sum(('lon','lat')))
    co2_aut.append(f2.sel(time=f2.time.dt.season=="SON").groupby('time.year').mean('time').sum(('lon','lat')))

### MPI-GE Mean

In [5]:
f3 = xr.open_dataset('/work/uo1075/from_Mistral/uo1075/u301331/observations/mpi_1990-2020_co2.nc')
f4 = f3.sel(depth=0).co2flux.sel(lon=slice(0,360),lat=slice(-65,-35)).sel(time=slice('1990','2019'))

#Mean Fluxes
mod_yr = f4.groupby('time.year').mean(dim='time').mean(dim='year')

seasons_mod = f4.groupby('time.season').mean(dim='time')
mod_DJF = seasons_mod.sel(season='DJF')/4
mod_MAM = seasons_mod.sel(season='MAM')/4
mod_JJA = seasons_mod.sel(season='JJA')/4
mod_SON = seasons_mod.sel(season='SON')/4

In [6]:
co2_ym.append((f4.groupby('time.year').mean('time')).sum(('lon','lat')))
co2_win.append(f4.sel(time=f4.time.dt.season=="DJF").groupby('time.year').mean('time').sum(('lon','lat')))
co2_spr.append(f4.sel(time=f4.time.dt.season=="MAM").groupby('time.year').mean('time').sum(('lon','lat')))
co2_sum.append(f4.sel(time=f4.time.dt.season=="JJA").groupby('time.year').mean('time').sum(('lon','lat')))
co2_aut.append(f4.sel(time=f4.time.dt.season=="SON").groupby('time.year').mean('time').sum(('lon','lat')))

In [7]:
co2_win_new = [x/4 for x in co2_win]
co2_spr_new = [x/4 for x in co2_spr]
co2_sum_new = [x/4 for x in co2_sum]
co2_aut_new = [x/4 for x in co2_aut]

In [8]:
data1_x = [obs_yr,obs_DJF,obs_MAM,obs_JJA,obs_SON]
data2_x = [mod_yr,mod_DJF,mod_MAM,mod_JJA,mod_SON]

### Analysis

In [9]:
obs_yr_lat=obs_yr.lat
obs_yr_lon=obs_yr.lon
obs_yr,  obs_yr_lon  = add_cyclic_point(obs_yr, coord=obs_yr_lon)

obs_DJF_lat=obs_DJF.lat
obs_DJF_lon=obs_DJF.lon
obs_DJF,  obs_DJF_lon  = add_cyclic_point(obs_DJF, coord=obs_DJF_lon)

obs_MAM_lat=obs_MAM.lat
obs_MAM_lon=obs_MAM.lon
obs_MAM,  obs_MAM_lon  = add_cyclic_point(obs_MAM, coord=obs_MAM_lon)

obs_JJA_lat=obs_JJA.lat
obs_JJA_lon=obs_JJA.lon
obs_JJA,  obs_JJA_lon  = add_cyclic_point(obs_JJA, coord=obs_JJA_lon)

obs_SON_lat=obs_SON.lat
obs_SON_lon=obs_SON.lon
obs_SON,  obs_SON_lon  = add_cyclic_point(obs_SON, coord=obs_SON_lon)


In [10]:
mod_yr_lat=mod_yr.lat
mod_yr_lon=mod_yr.lon
mod_yr,  mod_yr_lon  = add_cyclic_point(mod_yr, coord=mod_yr_lon)

mod_DJF_lat=mod_DJF.lat
mod_DJF_lon=mod_DJF.lon
mod_DJF,  mod_DJF_lon  = add_cyclic_point(mod_DJF, coord=mod_DJF_lon)

mod_MAM_lat=mod_MAM.lat
mod_MAM_lon=mod_MAM.lon
mod_MAM,  mod_MAM_lon  = add_cyclic_point(mod_MAM, coord=mod_MAM_lon)

mod_JJA_lat=mod_JJA.lat
mod_JJA_lon=mod_JJA.lon
mod_JJA,  mod_JJA_lon  = add_cyclic_point(mod_JJA, coord=mod_JJA_lon)

mod_SON_lat=mod_SON.lat
mod_SON_lon=mod_SON.lon
mod_SON,  mod_SON_lon  = add_cyclic_point(mod_SON, coord=mod_SON_lon)

### Figures

In [ ]:
#titles = ['Annual Mean','Winter Mean','Spring Mean','Summer Mean','Autumn Mean']
labels1=['a','b','c','d','e']
labels2=['f','g','h','i','j']

obs_lons=[obs_yr_lon,obs_DJF_lon,obs_MAM_lon,obs_JJA_lon,obs_SON_lon]
obs_lats=[obs_yr_lat,obs_DJF_lat,obs_MAM_lat,obs_JJA_lat,obs_SON_lat]
obs_data=[obs_yr,obs_DJF,obs_MAM,obs_JJA,obs_SON]

mod_lons=[mod_yr_lon,mod_DJF_lon,mod_MAM_lon,mod_JJA_lon,mod_SON_lon]
mod_lats=[mod_yr_lat,mod_DJF_lat,mod_MAM_lat,mod_JJA_lat,mod_SON_lat]
mod_data=[mod_yr,mod_DJF,mod_MAM,mod_JJA,mod_SON]

fig, axs = plt.subplots(2, 5, figsize=(18,6), sharex=True, sharey=True, dpi=600)

for i in range(0,5):
    ax          =plt.subplot(2,5,i+1, projection=ccrs.SouthPolarStereo(0, -90))
    
    ax.set_title(labels1[i], fontsize=10, loc='left', fontweight='semibold')
    
    p = plt.contourf(obs_lons[i], obs_lats[i],obs_data[i],
                 levels = np.linspace(-3e-4,3e-4,51),
                 transform      = ccrs.PlateCarree(),
                 cmap           = LinearSegmentedColormap.from_list("mycmap", list(zip(nodes, colors))), extend='both')
   
    ax.set_extent([-180,180,-35,-90], crs=ccrs.PlateCarree())

    #circle projection
    theta =np.linspace(0,2*np.pi, 100)
    center, radius = [0.5,0.5],0.5
    verts =np.vstack([np.sin(theta), np.cos(theta)]).T
    circle =mpath.Path(verts*radius+center)
    ax.set_boundary(circle, transform=ax
                    .transAxes)
    
    ax.add_feature(cfeature.GSHHSFeature(edgecolor='k',zorder=1,facecolor='lightgrey',linewidth=0.5))
    
for i in range(0,5):
    ax          =plt.subplot(2,5,i+6, projection=ccrs.SouthPolarStereo(0, -90))
    
    ax.set_title(labels2[i], fontsize=10, loc='left', fontweight='semibold')
    
    plt.contourf(mod_lons[i], mod_lats[i],mod_data[i],
                 levels = np.linspace(-3e-4,3e-4,51),
                 transform      = ccrs.PlateCarree(),
                 cmap           = LinearSegmentedColormap.from_list("mycmap", list(zip(nodes, colors))), extend='both')
    
    ax.set_extent([-180,180,-35,-90], crs=ccrs.PlateCarree())

    #circle projection
    theta =np.linspace(0,2*np.pi, 100)
    center, radius = [0.5,0.5],0.5
    verts =np.vstack([np.sin(theta), np.cos(theta)]).T
    circle =mpath.Path(verts*radius+center)
    ax.set_boundary(circle, transform=ax.transAxes)

    ax.add_feature(cfeature.GSHHSFeature(edgecolor='k',zorder=1,facecolor='lightgrey',linewidth=0.5))
    
plt.subplots_adjust(wspace=0.2, hspace=0.0)
plt.tight_layout()

plt.savefig('2023_Fig2Panela_1990_2020_MPI.svg', format="svg",transparent=True)

In [11]:
fig, axs = plt.subplots(1, 5, figsize=(18,2.5), dpi=600)

ax1          =plt.subplot(1,5,1)
    
for i in range(0,6):
    sns.kdeplot(co2_ym[i],fill=True, color='red', common_norm=False,
    alpha=.1, linewidth=0)
    
    sns.kdeplot(co2_ym[6], color='red')
    sns.kdeplot(co2_ym[7], color='black')

ax1.set_title('k', fontsize=10, loc='left', fontweight='semibold')

ax1.set_ylim(0,5)
ax1.set_xlim(-2,0)
ax1.axvspan(-2, 0, facecolor='dodgerblue', alpha=0.15)

ax1.set_yticks([0,5], fontsize=10)
ax1.set_xticks([-2,-1,0], fontsize=10)


ax2          =plt.subplot(1,5,2)
    
for i in range(0,6):
    sns.kdeplot(co2_win_new[i],fill=True, color='red', common_norm=False,
    alpha=.1, linewidth=0,)
    
    sns.kdeplot(co2_win_new[6], color='red')
    sns.kdeplot(co2_win_new[7], color='black')

ax2.set_title('l', fontsize=10, loc='left', fontweight='semibold')

plt.yticks([0,15], fontsize=10)
ax2.set_xticks([-1,-0.5, 0], fontsize=10)

ax2.set_ylim(0,15)
ax2.set_xlim(-1.1,0)
ax2.axvspan(0, -4, facecolor='dodgerblue', alpha=0.15)
plt.ylabel('')

ax3          =plt.subplot(1,5,3)
    
for i in range(0,6):
    sns.kdeplot(co2_spr_new[i],fill=True, color='red', common_norm=False, 
    alpha=.1, linewidth=0,)
    
    sns.kdeplot(co2_spr_new[6], color='red')
    sns.kdeplot(co2_spr_new[7], color='black')

ax3.set_title('m', fontsize=10, loc='left', fontweight='semibold')

plt.yticks([0,15], fontsize=10)
plt.xticks([-0.5,0,0.5], fontsize=10)

ax3.set_ylim(0,15)
ax3.set_xlim(-0.5,0.5)
ax3.axvspan(0, -2, facecolor='dodgerblue', alpha=0.15)
ax3.axvspan(0, 2, facecolor='orange', alpha=0.15)

plt.ylabel('')

ax4          =plt.subplot(1,5,4)
    
for i in range(0,6):
    sns.kdeplot(co2_sum_new[i],fill=True, color='red', common_norm=False, 
    alpha=.1, linewidth=0,)
    
    sns.kdeplot(co2_sum_new[6], color='red')
    sns.kdeplot(co2_sum_new[7], color='black')
    

ax4.set_title('n', fontsize=10, loc='left', fontweight='semibold')

plt.yticks([0,15], fontsize=10)
plt.xticks([-0.5,0,0.5], fontsize=10)

ax4.set_ylim(0,15)
ax4.set_xlim(-0.5,0.5)

ax4.axvspan(0, -2, facecolor='dodgerblue', alpha=0.15)
ax4.axvspan(0, 2, facecolor='orange', alpha=0.15)

plt.ylabel('')

ax5          =plt.subplot(1,5,5)
    
for i in range(0,6):
    sns.kdeplot(co2_aut_new[i],fill=True, color='red', common_norm=False,
    alpha=.1, linewidth=0,)
    
    sns.kdeplot(co2_aut_new[6], color='red', label='Multi-data Mean')
    sns.kdeplot(co2_aut_new[7], color='black',label='MPI-GE Mean')
    

ax5.set_title('o', fontsize=10, loc='left', fontweight='semibold')

plt.ylabel("")

plt.yticks([0,15], fontsize=10)
plt.xticks([-1,-0.5,0], fontsize=10)

ax5.set_ylim(0,15)
ax5.set_xlim(-1.1,0.0)
ax5.axvspan(0, -4, facecolor='dodgerblue', alpha=0.15)
ax5.axvspan(0, 0.25, facecolor='orange', alpha=0.15)

plt.savefig('2023_Fig2Panelb_1990_2020_MPI.svg', format="svg",transparent=True)

### Uncertainty estimates

In [12]:
# data1_x = [obs_yr,obs_DJF,obs_MAM,obs_JJA,obs_SON]
# data2_x = [mod_yr,mod_DJF,mod_MAM,mod_JJA,mod_SON]

mod_mean=np.zeros((5))
for i in range(0,5):
    mod_mean[i] = (data2_x[i].sum(('lon','lat')).values.round(3))
    
print(mod_mean.round(2))

obs_mean=np.zeros((5))
for i in range(0,5):
    obs_mean[i] = (data1_x[i].sum(('lon','lat')).values.round(3))
    
print(obs_mean.round(2))

[-1.1  -0.9   0.2   0.43 -0.82]
[-0.95 -0.29 -0.24 -0.2  -0.22]


In [13]:
model_yr_mean = np.zeros((100))
model_win_mean = np.zeros((100))
model_spr_mean = np.zeros((100))
model_sum_mean = np.zeros((100))
model_aut_mean = np.zeros((100))

for i in range(1,101):   
    f='{}{}{}'.format('/work/uo1075/from_Mistral/uo1075/u301331/model_GE_co2/GE_co2_',i,'.nc')
    f = xr.open_dataset(f)
    f1 = f.sel(depth=0).co2flux.sel(lon=slice(0,360),lat=slice(-65,-35)).sel(time=slice('1990','2019'))
    
    model_yr_mean[i-1]=(f1.sum(('lon','lat')).mean('time').values.round(3))
    model_win_mean[i-1]=(f1.sel(time=f1.time.dt.season=="DJF").mean('time').sum(('lon','lat')).values.round(3)/4)
    model_spr_mean[i-1]=(f1.sel(time=f1.time.dt.season=="MAM").mean('time').sum(('lon','lat')).values.round(3)/4)
    model_sum_mean[i-1]=(f1.sel(time=f1.time.dt.season=="JJA").mean('time').sum(('lon','lat')).values.round(3)/4)
    model_aut_mean[i-1]=(f1.sel(time=f1.time.dt.season=="SON").mean('time').sum(('lon','lat')).values.round(3)/4)

mod_range = ((np.abs(model_yr_mean).max()-np.abs(model_yr_mean).min())/2,
                  (np.abs(model_win_mean).max()-np.abs(model_win_mean).min())/2,
                  (np.abs(model_spr_mean).max()-np.abs(model_spr_mean).min())/2,
                  (np.abs(model_sum_mean).max()-np.abs(model_sum_mean).min())/2,
                  (np.abs(model_aut_mean).max()-np.abs(model_aut_mean).min())/2)
mod_range = np.round(mod_range,2)

# Mean values ± Range
print(str(np.round(np.mean(model_yr_mean[:]),decimals=2)) +u" \u00B1 "+ str(mod_range[0]) + 'Pg C/yr')
print(str((np.mean(model_win_mean[:])).round(2)) +u" \u00B1 " + str(mod_range[1]) +'Pg C/yr')
print(str((np.mean(model_spr_mean[:])).round(2)) +u" \u00B1 " + str(mod_range[2]) +'Pg C/yr')
print(str((np.mean(model_sum_mean[:])).round(2)) +u" \u00B1 " + str(mod_range[3]) +'Pg C/yr')
print(str((np.mean(model_aut_mean[:])).round(2)) +u" \u00B1 " + str(mod_range[4]) +'Pg C/yr')

-1.1 ± 0.03Pg C/yr
-0.9 ± 0.05Pg C/yr
0.2 ± 0.02Pg C/yr
0.43 ± 0.04Pg C/yr
-0.82 ± 0.03Pg C/yr


In [14]:
# Range for obs-based products 
co2_ym_mean =  [co2_ym[i].mean('year').values.round(3) for i in range(len(co2_ym[:6]))]
co2_win_mean = [co2_win[i].mean('year').values.round(3)/4 for i in range(len(co2_win[:6]))]
co2_spr_mean = [co2_spr[i].mean('year').values.round(3)/4 for i in range(len(co2_spr[:6]))]
co2_sum_mean = [co2_sum[i].mean('year').values.round(3)/4 for i in range(len(co2_sum[:6]))]
co2_aut_mean = [co2_aut[i].mean('year').values.round(3)/4 for i in range(len(co2_aut[:6]))]

obs_uncertainty = ((np.abs(co2_ym_mean).max()-np.abs(co2_ym_mean).min())/2,
                  (np.abs(co2_win_mean).max()-np.abs(co2_win_mean).min())/2,
                  (np.abs(co2_spr_mean).max()-np.abs(co2_spr_mean).min())/2,
                  (np.abs(co2_sum_mean).max()-np.abs(co2_sum_mean).min())/2,
                  (np.abs(co2_aut_mean).max()-np.abs(co2_aut_mean).min())/2)
obs_uncertainty = np.round(obs_uncertainty,2)

# Mean values ± Range
print(str(np.round(np.mean(co2_ym_mean),decimals=2)) +u" \u00B1 "+ str(obs_uncertainty[0]) + 'Pg C/yr')
print(str(np.round(np.mean(co2_win_mean),decimals=2)) +u" \u00B1 "+ str(obs_uncertainty[1]) + 'Pg C/yr')
print(str(np.round(np.mean(co2_spr_mean),decimals=2)) +u" \u00B1 "+ str(obs_uncertainty[2]) + 'Pg C/yr')
print(str(np.round(np.mean(co2_sum_mean),decimals=2)) +u" \u00B1 "+ str(obs_uncertainty[3]) + 'Pg C/yr')
print(str(np.round(np.mean(co2_aut_mean),decimals=2)) +u" \u00B1 "+ str(obs_uncertainty[4]) + 'Pg C/yr')

-0.93 ± 0.14Pg C/yr
-0.29 ± 0.06Pg C/yr
-0.23 ± 0.02Pg C/yr
-0.19 ± 0.03Pg C/yr
-0.22 ± 0.05Pg C/yr
